<a href="https://colab.research.google.com/github/rodolforpr14/Rodolfo-s-Works/blob/main/INDI_Alghoritm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Remove TGHD, do the expected application and Mark the Top ads (ads responsible to bring 80% of total expected application)**

In [1]:
import pandas as pd
import math
from google.colab import auth
import gspread
from google.auth import default
from sklearn.linear_model import LinearRegression
import numpy as np

auth.authenticate_user()

# Definir escopos
SCOPES = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Obter credenciais
creds, _ = default()
creds = creds.with_scopes(SCOPES)

# Verificar o e-mail da conta de serviço
service_account_email = creds.service_account_email if hasattr(creds, 'service_account_email') else "default"

# Autorização do gspread
gc = gspread.authorize(creds)

# Abrir a planilha do Google Sheets
spreadsheet= gc.open('Backstage 2.0 - INDI')  # Substitua pelo nome da sua planilha

worksheet = spreadsheet.worksheet("Filtered Data[Data-Base]")  # Ou use .worksheet('nome da aba') se não for a primeira aba

# Carregar os dados em um DataFrame
data = worksheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

df=df.drop(columns = ['Tier','Semana publicación'])
df.rename(columns={'JD Sheet Seniority': 'Seniority'}, inplace=True)
df['Title from JD Sheet'] = df['Title from JD Sheet'].astype(str)
df = df.loc[~df['Title from JD Sheet'].str.contains('TGHD', case=False)]
df['Total Postulados por oferta'] = pd.to_numeric(df['Total Postulados por oferta'], errors='coerce')
df['Applications Expected'] = df.groupby(['Technology', 'Seniority','Title from JD Sheet','Location'])['Total Postulados por oferta'].transform('mean')
df= df.drop(columns=['Total Postulados por oferta'])
df = df.dropna().copy()  # Corrigindo o aviso SettingWithCopyWarning

df['Applications Expected'] = pd.to_numeric(df['Applications Expected'], errors='coerce')
# Arredondando para cima
df['Applications Expected'] = df['Applications Expected'].apply(math.ceil)

df = df.astype({'Applications Expected': int}).copy()

df = df.drop_duplicates(subset=['Technology', 'Seniority','Title from JD Sheet','Location'])

df_sorted = df.sort_values(by=['Seniority', 'Technology', 'Applications Expected'], ascending=[True, True, False])
df_sorted['80% of Total Expected Applications'] = df_sorted.groupby(['Technology', 'Seniority'])['Applications Expected'].transform('sum') *0.8
results = []
for (technology, seniority), group in df_sorted.groupby(['Technology', 'Seniority']):
    total_applicants = group['80% of Total Expected Applications'].iloc[0]
    cumulative_sum = 0
    ads_count = 0

    # Iterar sobre o grupo ordenado por Total Applicants decrescente
    for index, row in group.sort_values(by='Applications Expected', ascending=False).iterrows():
        cumulative_sum += row['Applications Expected']
        ads_count += 1
        if cumulative_sum >= total_applicants:
            break

    # Armazenar os resultados
    results.append({
        'Technology': technology,
        'Seniority': seniority,
        '80% total applications Per Cluster': total_applicants,
        'Ads to Reach 80% of Expected applications Per Cluster': ads_count
    })

# Criar um DataFrame com os resultados
result_df = pd.DataFrame(results)

def mark_top_ads(df, result_df):
    df['Top Ads'] = 'ads_bellow'

    for index, row in result_df.iterrows():
        technology = row['Technology']
        seniority = row['Seniority']
        top_ads_count = row['Ads to Reach 80% of Expected applications Per Cluster']

        group = df[(df['Technology'] == technology) & (df['Seniority'] == seniority)]
        group = group.sort_values(by='Applications Expected', ascending=False)

        for i in range(top_ads_count):
            df.loc[group.index[i], 'Top Ads'] = 'top_ads'

    return df

# Chamar a função para marcar os top ads
df = mark_top_ads(df_sorted, result_df)
df= df.drop(columns=['80% of Total Expected Applications'])
df.to_csv('Test>.csv',index=False)
def new_combinations(df, existing_combinations):
    # Crie um conjunto de combinações existentes para verificação rápida
    existing_combinations_set = set(map(tuple, existing_combinations.values))

    combinations = []

    # Agrupar o DataFrame por tecnologia e senioridade
    for (technology, seniority), group_df in df.groupby(['Technology', 'Seniority']):
        unique_locations = set(group_df['Location'].unique())
        unique_titles = set(group_df['Title from JD Sheet'].unique())

        # Adicionar localidades correspondentes à tecnologia ao conjunto de localidades únicas
        technology_locations = set(df[df['Technology'] == technology]['Location'].unique())
        unique_locations |= technology_locations  # União dos conjuntos

        for location in unique_locations:
            for title in unique_titles:
                new_combination = {'Technology': technology, 'Seniority': seniority, 'Location': location, 'Title from JD Sheet': title}
                if tuple(new_combination.values()) not in existing_combinations_set:
                    combinations.append(new_combination)

    return pd.DataFrame(combinations)

# Supondo que df_sorted seja seu DataFrame com a coluna 'Top Ads' já marcada
existing_combinations = df_sorted[['Technology', 'Seniority', 'Location', 'Title from JD Sheet']].copy()

new_combinations_df = new_combinations(df_sorted, existing_combinations)

# Salvar ou imprimir o DataFrame de novas combinações
# new_combinations_df.to_csv("new_combinations.csv", index=False)

import math
spreadsheet= gc.open('Backstage 2.0 - INDI')  # Substitua pelo nome da sua planilha

worksheet = spreadsheet.worksheet("Filtered Data[Storage]")  # Ou use .worksheet('nome da aba') se não for a primeira aba

# Carregar os dados em um DataFrame
data = worksheet.get_all_values()

data_base = pd.DataFrame(data[1:], columns=data[0])

data_base = data_base.drop(columns=['Tier', 'Semana publicación'])
data_base.rename(columns={'JD Sheet Seniority': 'Seniority'}, inplace=True)
data_base['Title from JD Sheet'] = data_base['Title from JD Sheet'].astype(str)
data_base = data_base.loc[~data_base['Title from JD Sheet'].str.contains('TGHD', case=False)]
data_base['Total Postulados por oferta'] = pd.to_numeric(data_base['Total Postulados por oferta'], errors='coerce')
data_base['Applications Expected'] = data_base.groupby(
    ['Technology', 'Seniority', 'Title from JD Sheet', 'Location']
)['Total Postulados por oferta'].transform('mean')
data_base = data_base.dropna().copy()  # Corrigindo o aviso SettingWithCopyWarning
data_base['Applications Expected'] = pd.to_numeric(data_base['Applications Expected'], errors='coerce').apply(math.ceil)
data_base = data_base.astype({'Applications Expected': int}).copy()
data_base = data_base.drop(columns=['Total Postulados por oferta'])
data_base = data_base.drop_duplicates(subset=['Technology', 'Seniority', 'Title from JD Sheet', 'Location'])

def good_ads_storage(data_base):
    filtered_ads = pd.DataFrame(columns=data_base.columns)

    for (technology, seniority), group_data_base in data_base.groupby(['Technology', 'Seniority']):
        group_sorted = group_data_base.sort_values(by='Applications Expected', ascending=False)
        total_rows = len(group_sorted)

        split_index = math.ceil(total_rows * 0.8)
        filtered_group = group_sorted.iloc[:split_index].sort_values(by='Applications Expected', ascending=False)
        filtered_group['Type'] = 'good_ad'
        filtered_ads = pd.concat([filtered_ads, filtered_group])

    return filtered_ads

good_ads_per_cluster = good_ads_storage(data_base)

spreadsheet= gc.open('Backstage 2.0 - INDI')  # Substitua pelo nome da sua planilha

worksheet = spreadsheet.worksheet("Database Combinations")  # Ou use .worksheet('nome da aba') se não for a primeira aba

# Carregar os dados em um DataFrame
data = worksheet.get_all_values()

df3 = pd.DataFrame(data[1:], columns=data[0])
#columns_to_remove = ['Locations', 'Concat']
#df3 = df3.drop(columns=columns_to_remove)
df3.rename(columns={'Tech': 'Technology', 'Job Title': 'Title from JD Sheet'}, inplace=True)

def combinations_to_use(df3, new_combinations_df, df, good_ads_per_cluster):
    # Criar um conjunto de combinações existentes para evitar duplicatas
    existing_combinations_set = set(map(tuple, new_combinations_df[['Technology', 'Seniority', 'Location', 'Title from JD Sheet']].values))
    existing_combinations_set |= set(map(tuple, df[['Technology', 'Seniority', 'Location', 'Title from JD Sheet']].values))
    existing_combinations_set |= set(map(tuple, good_ads_per_cluster[['Technology', 'Seniority', 'Location', 'Title from JD Sheet']].values))

    # Lista para armazenar novas combinações
    combinations = []

    # Obter localizações únicas presentes em df e good_ads_per_cluster
    unique_locations_df = df['Location'].unique()
    unique_locations_good_ads = good_ads_per_cluster['Location'].unique()
    unique_locations = set(unique_locations_df).union(set(unique_locations_good_ads))

    # Iterar sobre cada linha do DataFrame df3
    for index, row in df3.iterrows():
        technology = row['Technology']
        seniority = row['Seniority']
        title = row['Title from JD Sheet']

        # Gerar todas as combinações possíveis de localização e título
        for location in unique_locations:
            new_combination = {
                'Technology': technology,
                'Seniority': seniority,
                'Location': location,
                'Title from JD Sheet': title
            }
            # Verificar se a nova combinação já existe no conjunto existente
            if tuple(new_combination.values()) not in existing_combinations_set:
                combinations.append(new_combination)

    # Retornar as novas combinações como um DataFrame
    return pd.DataFrame(combinations)

# Exemplo de uso
existing_combinations = new_combinations_df[['Technology', 'Seniority', 'Location', 'Title from JD Sheet']].copy()
combinations_to_use_df = combinations_to_use(df3, new_combinations_df, df, good_ads_per_cluster)

spreadsheet= gc.open('Backstage 2.0 - INDI')  # Substitua pelo nome da sua planilha

worksheet = spreadsheet.worksheet("Input Allocation")  # Ou use .worksheet('nome da aba') se não for a primeira aba

# Carregar os dados em um DataFrame
data = worksheet.get_all_values()

total_ads_per_cluster_df= pd.DataFrame(data[1:], columns=data[0])

def combine_ads(total_ads_per_cluster_df, df, new_combinations_df, combinations_to_use_df, good_ads_per_cluster):
    combined_ads_dict = {}

    # Garantir que a coluna 'Ads' seja do tipo inteiro
    total_ads_per_cluster_df['Ads'] = total_ads_per_cluster_df['Ads'].astype(int)

    for index, row in total_ads_per_cluster_df.iterrows():
        technology = row['Technology']
        seniority = row['Seniority']
        total_ads = row['Ads']

        ads_list = []

        if total_ads > 1:
            # Adicionar um anúncio do combinations_to_use_df com localidade presente em good_ads ou top_ads
            localities_from_ads = df[(df['Technology'] == technology) &
                                     (df['Seniority'] == seniority) &
                                     (df['Top Ads'] == 'top_ads')]['Location'].unique()
            localities_from_good_ads = good_ads_per_cluster[(good_ads_per_cluster['Technology'] == technology) &
                                                             (good_ads_per_cluster['Seniority'] == seniority)]['Location'].unique()
            all_localities = set(localities_from_ads) | set(localities_from_good_ads)
            additional_ad = combinations_to_use_df[(combinations_to_use_df['Technology'] == technology) &
                                                    (combinations_to_use_df['Seniority'] == seniority) &
                                                    (combinations_to_use_df['Location'].isin(all_localities)) &
                                                    (~combinations_to_use_df['Title from JD Sheet'].isin(df['Title from JD Sheet']))].head(1)
            if not additional_ad.empty:
                ads_list.append(additional_ad)
                total_ads -= 1  # Ajustar total_ads para refletir o número restante de anúncios necessários

        elif total_ads == 1:
            # Adicionar um anúncio do top_ads
            top_ads = df[(df['Technology'] == technology) & (df['Seniority'] == seniority) & (df['Top Ads'] == 'top_ads')]
            if not top_ads.empty:
                ads_list.append(top_ads.head(1))
                total_ads -= 1  # Ajustar total_ads para refletir o número restante de anúncios necessários

        # Filtrar os anúncios marcados como "top_ads" caso total_ads > 1
        top_ads = df[(df['Technology'] == technology) & (df['Seniority'] == seniority) & (df['Top Ads'] == 'top_ads')]

        # Calcular o número de anúncios restantes após os "top_ads"
        if len(top_ads) > total_ads:
            top_ads = top_ads.head(total_ads)

        remaining_ads_count = total_ads - len(ads_list) - len(top_ads)

        if remaining_ads_count > 0:
            # Calcular a quantidade de anúncios de good_ads_per_cluster
            good_ads = math.floor(remaining_ads_count * 0.3)
            remaining_good_ads = good_ads_per_cluster[(good_ads_per_cluster['Technology'] == technology) &
                                                      (good_ads_per_cluster['Seniority'] == seniority) &
                                                      (~good_ads_per_cluster.index.isin(top_ads.index))].head(good_ads)

            # Calcular a quantidade de anúncios de new_combinations
            new_combinations_ads = remaining_ads_count - len(remaining_good_ads)
            remaining_new_combinations = new_combinations_df[(new_combinations_df['Technology'] == technology) &
                                                             (new_combinations_df['Seniority'] == seniority) &
                                                             (~new_combinations_df.index.isin(top_ads.index))].head(new_combinations_ads)

            # Adicionar remaining_good_ads e remaining_new_combinations à lista de anúncios
            if not remaining_good_ads.empty:
                ads_list.append(remaining_good_ads)
            if not remaining_new_combinations.empty:
                ads_list.append(remaining_new_combinations)

        # Combinar os anúncios selecionados
        if ads_list:
            combined_ads = pd.concat([top_ads] + ads_list, ignore_index=True)
            # Garantir que o número total de anúncios não exceda total_ads
            combined_ads = combined_ads.head(row['Ads'])
        else:
            combined_ads = top_ads

        combined_ads_dict[(technology, seniority)] = combined_ads

    combined_ads = pd.concat(combined_ads_dict.values(), ignore_index=True)
    combined_ads = combined_ads.drop_duplicates(subset=['Title from JD Sheet', 'Seniority', 'Technology', 'Location'])

    ad_counts_per_cluster = combined_ads.groupby(['Technology', 'Seniority']).size().reset_index(name='Count')

    # Adicionar anúncios do combinations_to_use_df para atingir o total desejado por cluster
    for index, row in total_ads_per_cluster_df.iterrows():
        technology = row['Technology']
        seniority = row['Seniority']
        total_ads = row['Ads']

        # Verificar quantos anúncios já existem
        current_count = ad_counts_per_cluster[(ad_counts_per_cluster['Technology'] == technology) &
                                               (ad_counts_per_cluster['Seniority'] == seniority)]['Count']
        current_count = current_count.iloc[0] if not current_count.empty else 0

        # Calcular a quantidade de anúncios faltantes
        missing_ads_count = total_ads - current_count

        if missing_ads_count > 0:
            # Selecionar anúncios adicionais do combinations_to_use_df
            additional_ads = combinations_to_use_df[(combinations_to_use_df['Technology'] == technology) &
                                                    (combinations_to_use_df['Seniority'] == seniority) &
                                                    (~combinations_to_use_df.index.isin(combined_ads.index))].head(missing_ads_count)
            if not additional_ads.empty:
                combined_ads = pd.concat([combined_ads, additional_ads], ignore_index=True)

    # Remover duplicatas finais
    combined_ads = combined_ads.drop_duplicates(subset=['Title from JD Sheet', 'Seniority', 'Technology', 'Location'])

    return combined_ads



# Exemplo de uso
combined_ads = combine_ads(total_ads_per_cluster_df, df, new_combinations_df, combinations_to_use_df, good_ads_per_cluster)
combined_ads = combined_ads.drop(columns=['Year','Month','Top Ads'])

combined_ads['Applications Expected'] = combined_ads['Applications Expected'].fillna(0)
# Substituir valores infinitos por 0
combined_ads.replace([float('inf'), float('-inf')], 0, inplace=True)

# Converter o DataFrame para uma lista de listas, incluindo o cabeçalho
data_to_update = [combined_ads.columns.values.tolist()] + combined_ads.values.tolist()

# Abrir a planilha e a worksheet
spreadsheet = gc.open('Backstage 2.0 - INDI')
worksheet2 = spreadsheet.worksheet("Output Generation Code")

# Atualizar a planilha com os dados combinados
worksheet2.update(data_to_update)

print(combined_ads)

                 Title from JD Sheet  \
0         Junior .NET Developer - EN   
1          .NET Junior Engineer - EN   
2         Junior .NET Developer - EN   
3                .NET Tech Lead - EN   
4                .NET Developer - EN   
..                               ...   
116   Quality Assurance Analyst - EN   
117   Quality Assurance Analyst - EN   
118        TypeScript Developer - EN   
119  User Experience Specialist - EN   
120  User Experience Specialist - EN   

                                             Seniority  \
0                  Junior / Assistant / Junior Analyst   
1                  Junior / Assistant / Junior Analyst   
2                  Junior / Assistant / Junior Analyst   
3    Lead / Leader / Manager / Tech Lead / Engineer...   
4                                Semi Senior / Analyst   
..                                                 ...   
116                              Semi Senior / Analyst   
117                              Semi Senior / Analyst 